# Poisson - Gamma Example

Let's say we're studying the number of customers visiting a coffee shop daily. We observe the number of customers for 10 days and want to infer the average number of daily visits.

We can model the daily number of visits as a Poisson distribution, where the rate parameter λ represents the average number of daily visits. Since λ is unknown, we put a Gamma prior on it. The Gamma distribution is a suitable prior for λ because it is always positive and can represent a wide range of shapes depending on its parameters.

## Base R

-   We think around 10 customers visit the shop per day, but we are not sure of that at all

    -   We want the expected value alpha/beta = 10

    -   But we want the standard deviation sqrt(alpha) / beta to be high too (maybe around 10?)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cmdstanpy import CmdStanModel, cmdstan_path
from scipy.stats import gamma, poisson, nbinom

# Prior hyperparameters
alpha_prior = 1
beta_prior = 0.1

# Visualize the prior distribution
prior_dist = gamma.rvs(alpha_prior, scale=1/beta_prior, size=1000000)
plt.clf()
plt.hist(prior_dist, bins=100)
plt.show()

-   I think this prior distribution adequately describes my beliefs, because I'm pretty sure that there are fewer than 20 customers per day.

In [ ]:
# 95% prior credible interval for lambda
gamma.ppf(q = [0.025, 0.975], a = alpha_prior, scale = 1/beta_prior)

-   We simulate data, but set a seed so it is reproducible.

-   This is the 10 days of data we observe from the coffee shop, where we observe between 2 and 9 customers per day.

In [ ]:
# Simulate some Poisson data
np.random.seed(123)  # for reproducibility
N = 10
true_lambda = 5
x = poisson.rvs(true_lambda, size=N)
print(x)

- We update the parameters and make a histogram of the posterior distribution.

In [ ]:
# Update parameters
alpha_posterior = alpha_prior + np.sum(x)
beta_posterior = beta_prior + N

# histogram of the posterior distribution
plt.clf()
plt.hist(gamma.rvs(alpha_posterior, scale=1/beta_posterior, size=100000), bins=100)
plt.show()

- Here is a 95% credible interval using qgamma.

In [ ]:
# 95% posterior credible interval for theta
gamma.ppf([0.025, 0.975], a=alpha_posterior, scale = 1/beta_posterior)

-  This is a predictive distribution for future observations, accounting for the inherent daily uncertainty and our uncertainty about the rate $\lambda$. 

In [ ]:
# Predictive distribution
lambda_draws = gamma.rvs(alpha_posterior, scale=1/beta_posterior, size=100000)
future_obs = poisson.rvs(lambda_draws)
plt.clf()
plt.hist(future_obs, bins=100)
plt.show()

- Here is the predictive distribution directly, which is a negative binomial distribution. 

In [ ]:
# Predictive distribution (Negative binomial)
param1 = alpha_posterior
param2 = beta_posterior/(beta_posterior + 1)
plt.clf()
plt.hist(nbinom.rvs(param1, param2, size=100000), bins=100)
plt.show()

# Stan

-   We need to load in the Stan packages and define the Poisson-Gamma model.

In [ ]:
# Define Stan model
stan_code = '''
data {
  int<lower=0> N; // number of observations
  int<lower=0> y[N]; // observed counts
  real<lower=0.1> alpha; // prior hyperparameter
  real<lower=0.1> beta; // prior hyperparameter
}
parameters {
  real<lower=0> lambda; // Poisson rate parameter
}
model {
  lambda ~ gamma(alpha, beta); // prior
  y ~ poisson(lambda); // likelihood
}
generated quantities {
  int y_pred;
  y_pred = poisson_rng(lambda); // posterior predictive distribution
}
'''

# Save model to a file and compile
with open("model.stan", "w") as file:
    file.write(stan_code)
model = CmdStanModel(stan_file="model.stan")

-   We need to simulate the data from above (this is still in my R environment from the example above without Stan, but it doesn't hurt to show it again!)

-   Also, we define the hyperparameters and package together all the data for Stan, and fit the model based on the data.

In [ ]:
# Define data for Stan model
data = {
    "N": N,
    "y": x,
    "alpha": alpha_prior,
    "beta": beta_prior
}

# Fit the model
fit = model.sample(data=data, chains=4)

-   We draw samples to determine our posterior distribution and use it to make a credible interval.


In [ ]:
# Draw samples to determine our posterior distribution
posterior_samples = fit.stan_variable('lambda')

# 95% credible interval
hpd = np.percentile(posterior_samples, [2.5, 97.5])
hpd

# Extract the point estimates
expected_value = np.mean(posterior_samples)
post_median = np.median(posterior_samples)
print(f"Expected value for lambda: {expected_value}\nPosterior median for lambda: {post_median}")

-   We can find the point estimates.

In [ ]:
# MAP estimate (not possible with continuous parameters, we would have to create some analytical 
# approximation and use that)

#map_estimate <- mode(posterior_samples_df$lambda)
#cat("MAP estimate for lambda: ", map_estimate, "\n")

# Posterior mean
expected_value = np.mean(posterior_samples)
print("Expected value for lambda: ", expected_value, "\n")

# Posterior median
post_median = np.median(posterior_samples)
print("Posterior median for lambda: ", post_median, "\n")

-   We can draw posterior samples (which follow a negative binomial distribution)

In [ ]:
# Posterior predictive samples
y_pred_samples = fit.stan_variable('y_pred')
plt.clf()
plt.hist(y_pred_samples, bins=100)
plt.show()